<font size="6"><b>`DATA.TABLE` PACKAGE</b></font>

In [ ]:
library(data.table)
library(nycflights13)
library(DT)

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=30) # for limiting the number of top and bottom rows of tables printed 

![xkcd](../imagesba/data_trap.png)

(https://xkcd.com/2582)

In this session, we will cover basic features of data.table

Important: Do not confuse data.table package for data wrangling and datatable function from DT package for creating interactive widgets to view data

Why data.table is useful? Hadley Wickham et al. says:

> If you’re routinely working with larger data (10-100 Gb, say), you should learn more about data.table. This book doesn’t teach data.table because it has a very concise interface which makes it harder to learn since it offers fewer linguistic cues. But if you’re working with large data, the performance payoff is worth the extra effort required to learn it.

(https://r4ds.had.co.nz/introduction.html)

# Datasets

You can get info on and preview the structure and some rows of the datasets and navigate through them

## airports

In [ ]:
#?airports

In [ ]:
head(airports)

In [ ]:
str(airports)

In [ ]:
datatable(airports, filter = "top")

## weather

In [ ]:
#?weather

In [ ]:
head(weather)

In [ ]:
str(weather)

In [ ]:
#datatable(weather, filter = "top")

## flights

In [ ]:
#?flights

In [ ]:
head(flights)

In [ ]:
str(flights)

In [ ]:
#datatable(flights, filter = "top")

In [ ]:
class(flights)

In [ ]:
attributes(flights)[setdiff(names(attributes(flights)), c("row.names", "index"))]

# convert data.frames to data.tables

`setDT` function converts an existing data.frame or similar object into a data.table in place (without explicit assignment so it is memory efficient) so that you can use the concise data.table syntax:

In [ ]:
airports <- copy(airports)
setDT(airports)

In [ ]:
weather <- copy(weather)
setDT(weather)

In [ ]:
class(flights)

In [ ]:
flights <- copy(flights)
setDT(flights)

In [ ]:
class(flights)

# basic operations

DT[i, j, by]

## i: filtering rows

- You don't have to put an additional "," if you just want to filter rows
- You don't have to repeat the object name with flights$xxx to refer to the columns

In [ ]:
flights[distance < 400 & dep_delay > 20]

In [ ]:
flights[!hour %between% c(7, 24)]

## j: column operations

You have to use an `i` filter to use column operations

If you want to take all rows, use a leading "," inside the brackets as such:

DT[, ...]

Column names:

In [ ]:
names(flights)

Extract a single column as vector:

In [ ]:
flights[, head(carrier)]

Combine filter and column operations

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
       carrier]

Extract a single column as data.table

`.()` is an alias for `list()`

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35, .(carrier)]

Extract multiple columns as data.table

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        .(carrier, origin, dest)]

Extract a range of columns:

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        dep_time:arr_delay]

A new data output with calculated columns that summarize the data

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        .(max_at = max(air_time), min_at = min(air_time))]

A new data output with calculated columns that matches the rows

For example numeric 517 in dep_time represents 5:17, so we have to separate into two columns for that

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        .(dep_hour = dep_time %/% 100, dep_min = dep_time %% 100)]

### In-place modification operator (:=) 

Calculate new columns, with a value for each row, and add to the dataset

Note the `:=` operator for in-place modification

We can do that for adding a new column or modifying an existing column

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        dep_hour := dep_time %/% 100]

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        dep_min := dep_time %% 100]

For filtered-out rows, columns will have NA values

In [ ]:
flights[!is.na(dep_min), .(dep_time, dep_hour, dep_min)]

In [ ]:
str(flights)

Delete existing columns:

In [ ]:
flights[, dep_min := NULL]
flights[, dep_hour := NULL]

Calculate and add multiple columns in a single step.

Note the wrapping () in the LHS and .() in the RHS

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        (c("dep_hour", "dep_min")) := .(dep_time %/% 100, dep_time %% 100)]

In [ ]:
flights[!is.na(dep_min), .(dep_time, dep_hour, dep_min)]

Add a summarizing column, summarization done using filtered rows

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        (c("max_at", "min_at", "av_at")) := .(max(air_time), min(air_time), mean(air_time))]

Note the values will be same across rows when a summarized column is added

In [ ]:
flights[distance < 400 & dep_delay > 20 & !hour %between% c(7, 24) & air_time < 35,
        .(max_at, min_at, av_at)]

We can make calculations with interim unsaved step and assign or return the last calculation, if we wrap multiple statements inside a curly pair `{}` and end each with a semicolon `;`

In [ ]:
flights[, z_at := { std_at <- sd(air_time, na.rm = T);
                    av_at <- mean(air_time, na.rm = T);
                    (air_time - av_at) / std_at
                   }
                   ]

In [ ]:
flights[, summary(z_at)]

In [ ]:
flights[, mean(z_at, na.rm = T)]

## by operations

Repeat operations for each unique value of selected column(s), create a new output

In [ ]:
flights[, .(max_at_o = max(air_time, na.rm = T), min_at_o = min(air_time, na.rm = T), av_at_o = mean(air_time, na.rm = T)), by = origin]

In [ ]:
flights[, .(max_at_oc = max(air_time, na.rm = T), min_at_oc = min(air_time, na.rm = T), av_at_oc = mean(air_time, na.rm = T)),
        by = c("origin", "carrier")]

Or we can add the column back to the dataset, values will be calculated separately for each unique values of selected column(s)

Note that linebreaks are arbitrary, but as a style guidelines, try to keep the widgth of lines shorter for readability

In [ ]:
flights[, (c("max_at_oc", "min_at_oc", "av_at_oc")) := .(max(air_time, na.rm = T),
                                                         min(air_time, na.rm = T),
                                                         mean(air_time, na.rm = T)),
        by = c("origin", "carrier")]

In [ ]:
flights[seq(1e4, 1e5, 1e4), .(origin, carrier, max_at_oc, min_at_oc, av_at_oc)]

# chaining

We can make successive operations chaining braces

Note that in-place modification operation is quiet. To return the last output use a last empty chain with `[]`

In [ ]:
flights[, (c("std_at", "av_at")) := .(sd(air_time, na.rm = T),
                                      mean(air_time, na.rm = T))][,
    z_at := (air_time - av_at) / std_at][]

# ordering and naming

## setorder

setorder sets the row order according to columns and assigns in place

In [ ]:
head(flights)

In [ ]:
setorder(flights, carrier, time_hour)

In [ ]:
head(flights)

## setnames

Changes the names of columns

In [ ]:
setnames(flights, "time_hour", "date_time")

In [ ]:
flights

In [ ]:
setnames(flights, "date_time", "time_hour")

In [ ]:
flights

# symbols and shortcuts

## .SD

To refer to all columns or repeat operations on each column, you can use .SD

You may get the first flights of each day:

In [ ]:
flights[, .SD[1], by = c("year", "month", "day")]

Get the classes of all columns:

In [ ]:
flights[, lapply(.SD, class)]

Get the min max ranges of selected columns

In [ ]:
flights[, .(max_at_oc, min_at_oc, av_at_oc)][, lapply(.SD, range)]

## .SDcols

We may want to repeat calculations on or refer to multiple columns but not all of them

.SDcols makes a selection of columns that .SD will refer to

In [ ]:
flights[, lapply(.SD, range), .SDcols = c("max_at_oc", "min_at_oc", "av_at_oc")]

In [ ]:
flights[, .SD[1], by = c("year", "month", "day"), .SDcols = c("dep_time", "arr_time")]

## .N

Gives the number of filtered or grouped rows

In [ ]:
nrow(flights)

In [ ]:
flights[, .N]

In [ ]:
flights[month == 1, .N]

This returns the last row:

In [ ]:
flights[.N]

Number of rows by carrier

In [ ]:
flights[, .N, by = carrier]

Or along with the shares in total

In [ ]:
flights[, .N, by = carrier][, nshare := N / sum(N)][]

Or create an index column separately for each unique value combinations of selected columns

In [ ]:
flights[, index1 := 1:.N, by = c("carrier", "origin")]

In [ ]:
flights

## .I

.I refers to the index of rows.

There is only a single index for the filtered rows, not separate for each group according to the `by`

In [ ]:
flights[, index2 := .I]

In [ ]:
flights[, index3 := .I, by = c("carrier", "origin")]

In [ ]:
flights

In [ ]:
flights[, identical(index2, index3)]

# reshaping

## dcast

Let's revisit the example where we calculated the summaries of air time for each origin and carrier

In [ ]:
flights_at <- flights[, .(max_at_oc = max(air_time, na.rm = T), min_at_oc = min(air_time, na.rm = T), av_at_oc = mean(air_time, na.rm = T)),
        by = c("origin", "carrier")]

In [ ]:
flights_at

Now we may want to see the average values for each origin and carrier in a grid form

In [ ]:
flights_at_wide <- dcast(flights_at, carrier ~ origin, value.var = "av_at_oc")

In [ ]:
flights_at_wide

## melt

Now let's make this wide format into a long again

In [ ]:
melt(flights_at_wide, id.vars = "carrier", variable.name = "origin2", value.name = "av_time2")

# APPENDIX: Other useful features and functions

## split and rbindlist

Sometimes we may want to split the larger data.table into a list of smaller ones, split along a column(s).

Let's split `flights` across `origin` column:

In [ ]:
flights_list <- split(flights, by = "origin")

In [ ]:
str(flights_list)

And combine data.tables in a list into a larger whole again:

In [ ]:
flights <- rbindlist(flights_list, fill = T)

In [ ]:
str(flights)

## froll

frollmean calculates moving averages for specified window length.

For example, the moving average air time of the last flights between each origin and destination:

In [ ]:
flights[, air_time_av5 := frollmean(air_time, 5, na.rm = T), by = c("origin", "dest")]

In [ ]:
flights[origin == "EWR" & dest == "IAH", .(air_time, air_time_av5)]

frollsum does the same thing for sums

frollapply applies an arbitrary numeric function on a sliding window of specified length.

Let's repeat the sliding window calculation to get the maximum air time of the last 5 flights between origin and destination:

In [ ]:
flights[, air_time_max5 := frollapply(air_time, 5, max), by = c("origin", "dest")]

In [ ]:
flights[origin == "EWR" & dest == "IAH", .(air_time, air_time_av5, air_time_max5)]

In [ ]:
flights[, (c("air_time_av5", "air_time_max5")) := NULL]

## nafill

nafill takes fixed values or previous/following non-missing values to fill missing ones:

In [ ]:
flights[is.na(air_time), .(time_hour, carrier, origin, dest, air_time)]

Last observation can be carried forward:

In [ ]:
flights[, air_time_filled := nafill(air_time, type = "locf"), by = c("origin", "dest")]

In [ ]:
flights[is.na(air_time), .(time_hour, carrier, origin, dest, air_time, air_time_filled)]

Or next observation can be carried back:

In [ ]:
flights[, air_time_filled := nafill(air_time, type = "nocb"), by = c("origin", "dest")]

In [ ]:
flights[is.na(air_time), .(time_hour, carrier, origin, dest, air_time, air_time_filled)]

Or a constant value - for example the average non-missing air times of the origin destination pair - can be used to impute missing values:

In [ ]:
flights[, air_time_filled := nafill(air_time, type = "const", fill = mean(air_time, na.rm = T)), by = c("origin", "dest")]

In [ ]:
flights[is.na(air_time), .(time_hour, carrier, origin, dest, air_time, air_time_filled)]

In [ ]:
flights[, air_time_filled := NULL]

## shift

Shift can be used in order to create columns for lagging or leading values for single or multiple offsets:

Create five columns for previous five air times between each origin and destination.

In [ ]:
flights[, (paste("air_time_lag_", 1:5, sep = "")) := shift(air_time, 1:5, type = "lag"), by = c("origin", "dest")]

In [ ]:
flights[origin == "EWR" & dest == "IAH", .SD,
        .SDcols = c("origin", "dest", "air_time", paste("air_time_lag_", 1:5, sep = ""))]

In [ ]:
flights[, (paste("air_time_lag_", 1:5, sep = "")) := NULL]

Create five columns for next 5 air times between each origin and destination.

In [ ]:
flights[, (paste("air_time_lead_", 1:5, sep = "")) := shift(air_time, 1:5, type = "lead"), by = c("origin", "dest")]

In [ ]:
flights[origin == "EWR" & dest == "IAH", .SD,
        .SDcols = c("origin", "dest", "air_time", paste("air_time_lead_", 1:5, sep = ""))]

## setcolorder

Changes the order of columns

In [ ]:
setcolorder(flights, c("time_hour", "carrier")) 

In [ ]:
flights

## .GRP

Unique id number for each group according to `by`

In [ ]:
flights[, grp1 := .GRP, by = c("carrier", "origin")]

In [ ]:
flights[, grp1[1], by = c("carrier", "origin")]

## rleid

Unique id for contigious same values

In [ ]:
setorder(flights, origin, time_hour)

In [ ]:
flights[, rl_co := rleid(carrier), by = origin]

In [ ]:
flights[, .(time_hour, origin, carrier, rl_co)]

## joins

### Exact merge

Merge the coordinates of origins using merge()

In [ ]:
merge(flights[, .(origin, dest, distance)], airports[, .(faa, lat, lon)], by.x = "origin", by.y = "faa")

Or using the right join method: B[A, on ...]

In [ ]:
airports[, .(faa, lat, lon)][flights[, .(origin, dest, distance)], on = c(faa = "origin")]

### roll merge

Suppose we want to join the wind speed as at the time of departure for each origin

Now first calculate the exact date time of departure by adding the minutes of departure time in seconds:

In [ ]:
flights[, time_hour2 := time_hour + ifelse(is.na(dep_time), 30, dep_time) %% 100 * 60]

In [ ]:
flights

Now let's do exact merge but a great majority of rows do not match:

In [ ]:
weather[, .(origin, time_hour, wind_speed)][flights[, .(time_hour2, origin, carrier, flight)],
                                            on = c("origin", time_hour = "time_hour2")]

In [ ]:
weather

Now let's match the closest wind speed reading before the flight if there is no exact match:

In [ ]:
weather[, .(origin, time_hour, time_hour_org = time_hour, wind_speed)][flights[, .(time_hour2, origin, carrier, flight)],
                                            on = c("origin", time_hour = "time_hour2"), roll = Inf]

Or the closest wind speed reading after the flight if there is no exact match:

In [ ]:
weather[, .(origin, time_hour, wind_speed)][flights[, .(time_hour2, origin, carrier, flight)],
                                            on = c("origin", time_hour = "time_hour2"), roll = -Inf]

# Object Generating Code

In [ ]:
student_id <- 2025000000
library(data.table)
library(stringr)
seedx <- student_id + 123
set.seed(seedx)
nobs <- 1e3
ncat <- 2
nnum <- 4
nlevs <- sample(3:6, ncat, prob = 6:3)
namesfac <- paste(sample(words, ncat), "1", sep = "")
namesnum <- paste(sample(words, nnum), "1", sep = "")
levs <- lapply(nlevs, function(x) sample(letters, x))
facvals <- lapply(levs, function(x) sample(x, nobs, replace = T))
facvals <- mapply(function(x, y) factor(x, levels = y), facvals, levs, SIMPLIFY = F)
nummeans <- rnorm(nnum, 3, 4)
numsds <- rexp(nnum, 1/3)
numvals <- mapply(function(x, y) rnorm(nobs, x, y), nummeans, numsds, SIMPLIFY = F)
dt1 <- c(facvals, numvals)
setDT(dt1)
setnames(dt1, c(namesfac, namesnum))
options(repr.matrix.max.rows=20, repr.matrix.max.cols=30) # for limiting the number of top and bottom rows of tables printed 

Let's view our data:

In [ ]:
head(dt1)

In [ ]:
str(dt1)

## Tutorial

### filtering rows

Filter only those rows where mark1 variable takes the value of "m":

In [ ]:
dt1[before1 > 8 & play1 < 0.8]

### column operations

Report a new variable "somenewname1" that squares "member1" and subtracts "before1":

In [ ]:
dt1[, .(somenewname1 = member1^2 - before1)]

### In-place modification

Create a new variable "somenewname2" by dividing "play1" with "maybe1" and assign it back to dt1:

In [ ]:
dt1[, somenewname2 := play1 / maybe1]

In [ ]:
dt1

### by operations

Report a new variable "somenewname3" which holds the mean value of "member1" for each distinct value of "mark1":

In [ ]:
dt1[, .(somenewname3 = mean(member1)), by = mark1]

### casting

Create a new data.table names "somenewname4" which has the unique values of mark1 in its rows, unique values of create1 in its columns and calculates the maximum values of before1 for the rows in dt1 that contains corresponding unique values of mark1 and create1:

In [ ]:
somenewname4 <- dcast(dt1, mark1 ~ create1, fun.aggregate = max, value.var = "before1")

In [ ]:
somenewname4

### melting

Take somenewname4 data.table object, hold "mark1" as the ID variable, melt the object so that column names in "somenewname4" are collected in a new column called "somenewname5" and the values in "somenewname4" are collected in another column called "somenewname6":

In [ ]:
melt(somenewname4, id.vars = "mark1", variable.name = "somenewname5", value.name = "somenewname6")